# Calculate Q-Rookley

- Input data "20172022_processed_1_3_4.csv", from 2017/7/1 to 2022/12/31, same as used in Figlewski

- Use 500 rolling window for P density (when calculating EPK)

- Use logreturn instead of log+1

- Use 
```
r.source('Q_Rookley_main_3.R')
```
and
```
'EPK_library_1.r'
```
to calculate Q_Rookley, along with 
```
get_btc_prices_2015_2022
```
for physical density.

- Interest rate: 0

- Moneyness range is [min moneyness, max moneyness]

- Bandwidth 0.3

- tau=39

In [1]:
import platform
plat = platform.system()
import locale
if plat == 'Darwin':
    print('Using Macos Locale')
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
else:
    locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import datetime
import pandas as pd
import sys
import matplotlib.pyplot as plt
import csv
import gc
import math
import numpy as np
import os
import pdb
import pickle
import statsmodels.api as sm
import time


import rpy2.robjects.packages as rpackages
from rpy2 import robjects

from scipy import stats
from scipy.integrate import simps
from scipy import interpolate

from src.helpers import save_dict, load_dict
from src.brc import BRC
from src.strategies import IronCondor
from src.quandl_data import get_btc_prices_2015_2022

from numpy.lib.function_base import insert

from statsmodels.nonparametric.kernel_regression import KernelReg as nadarajawatson

from sklearn.neighbors import KernelDensity

from pathlib import Path

def filter_sub(_df,  mindate, maxdate, tau):
    # Subset

    # Only calls, tau in [0, 0.25] and fix one day (bc looking at intra day here)
    # (_df['is_call'] == 1) & --> Dont need to filter for calls because that happens in the R script
    # Also need to consider put-call parity there
    sub = _df[(_df['date'] >= mindate) & (_df['date'] <= maxdate) & (_df['moneyness'] >= 0.5) & (_df['moneyness'] < 2) &(_df['mark_iv'] > 0)]# &

    if tau > 0:
        sub = sub[sub['tau'] == tau]

    nrows = sub.shape[0]
    if nrows == 0:
        raise(ValueError('Sub is empty'))
   
    sub['moneyness'] = round(sub['moneyness'], 3)
    sub['underlying_price'] = round(sub['underlying_price'], 2)

    sub = sub.drop_duplicates()
    
    print(sub.describe())

    if nrows > 50000:
        print('large df, subsetting')
        sub = sub.sample(10000)
        print(sub.describe())

    return sub

def prepare_confidence_band_data(df):

            # Old
                # Save for Confidence Band 
                    # Column 1 - Day
                    # Column 2 - Month
                    # Column 3 - Year
                    # Column 4 - Call Indicator
                    # Column 5 - Time Maturity in Days
                    # Column 6 - Strike
                    # Column 7 - IV
                    # Column 8 - Spot
                    # Column 9 - Risk Free Rate
                    # Column 10 - 
                    # Column 11 - Forward
                # 3	1	1997	0	49	3025	165.8	2861.706	0.031465	0.150026951	2863.26
                # Day Month Year Put Days Strike IV     Spot         Risk Free    Tau in Days (255)       Forward

    """
    # Mapping Table for Input

    # Column ... X
    # 1 - date
    # 2 - IV
    # 3 - 1 for Call, 0 for Put
    # 4 - Tau 
    # 5 - Strike
    # 6 - assigned meanwhile as put2call[,6]+ put2call[,7] -  put2call[,5]*exp(- mean(put2call[,8])* tau);
    # 7 - Spot
    # 8 - Probably Interest Rate
    # 9 - assigned meanwhile Moneyness: Spot/Strike
    """
    b = df[['date', 'mark_iv', 'putcall', 'tau', 'K', 'underlying_price', 'interest_rate', 'moneyness', 'instrument_price']]
    b['placeholder'] = 0
    b = b.reset_index(drop=True)
    # b['interest_rate'] = 0.00
    b['forward'] = 1

    # Reorder
    df_c = b[['date', 'mark_iv', 'putcall', 'tau', 'K', 'instrument_price', 'underlying_price', 'interest_rate', 'moneyness']]

    unique_maturities = df.days_to_maturity.unique()[0]
    fname = str('confidence_band_input_') + str(unique_maturities) + str('.csv')
    df_c.to_csv(fname, index = False)
    return fname

def bootstrap(conf_fname, phys_fname, rdate, tau, simmethod, r, out_dir, bw, lower, upper):
                            
    try:
        # MAKE SURE TO USE DIFFERENT 2ND INPUT FOR SP500!!
        moneyness, spd, epk, epk_lo, epk_up, logret, spd_logret, epk_logret, epk_logret_lo, epk_logret_up, epk_BS, tau, volas, spd_lo, spd_up, cdf_m, cdf_ret= r.bootstrap_epk(robjects.StrVector([conf_fname]), 
                                      robjects.StrVector([phys_fname]),
                                      robjects.StrVector(rdate),
                                      robjects.StrVector([out_dir]),
                                      robjects.FloatVector([bw]),
                                      robjects.FloatVector([lower]),
                                      robjects.FloatVector([upper])
                                     )
        # Can recover PD here as SPD/EPK
        spd_df = pd.DataFrame({'m': moneyness,
                               'spdy': spd,
                               'spdy_up': spd_up,
                               'spdy_dn': spd_lo,
                               'epk': epk,
                               'ret': logret,
                               'spd_ret': spd_logret,
                               'cb_ret_up': epk_logret_up,
                               'cb_ret_dn': epk_logret_lo,
                               'epk_ret': epk_logret,
                               'volatility': volas,
                               'epk_up': epk_up,
                               'epk_dn': epk_lo,
                               'cdf_m': cdf_m,
                               'cdf_ret': cdf_ret})

        return spd_df
    
    except Exception as e:
        print('Sim failed, proceeding with the next one... - ', simmethod)
        print('exception: ', e)
        
# Initiate R Objects
base = rpackages.importr("base") 
r = robjects.r
r.source('src/confidence_bands.R')
r.source('src/Q_Rookley_main_3.R')

# BTC prices
tdat = get_btc_prices_2015_2022()
tdat[::-1].to_csv('Data/BTC_USD_Quandl_2015_2022.csv', index = False)

bw_range = np.array([0.3])
r_bandwidth = bw_range[0]

simmethod = ['oldschool']

df = pd.read_csv('Data/20172022_processed_1_3_4.csv')
interest_rate_data = pd.read_csv('Data/IR_daily.csv')
interest_rate_data = interest_rate_data.rename(columns={'index': 'date','DTB3': 'interest_rate'})

df1=df[df['tau']==39]
dates=df1['date'].unique()
#df1=df[df['date']>=dates[9]]
#dates = df1['date'].unique()

dates1 = dates

error_i = []
for i0 in range(dates1.size):
    df2 = df1[df1['date']==dates1[i0]]
    
    curr_day = datetime.datetime.strptime(dates1[i0],'%Y-%m-%d')
    curr_day_starttime = curr_day.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
    curr_day_endtime = curr_day.replace(hour = 23, minute = 59, second = 59, microsecond = 0)
    print(curr_day)

    df3 = pd.merge(df2, interest_rate_data, on='date', how='left')
    

    df3['interest_rate'] = 0
    df3['days_to_maturity'] = df3['tau']
    df3['tau'] = df3['tau']/365
    df3['underlying_price'] = df3['BTC_price']
    df3['instrument_price'] = df3['option_price']
    df3['putcall'][df3['putcall']=='C']=1
    df3['putcall'][df3['putcall']=='P']=0
    
    df_1 = df3[['date','BTC_price', 'K', 'interest_rate', 'tau', 'putcall', 'IV',  'moneyness',
                'days_to_maturity',  'underlying_price', 'instrument_price']]
    df_1['mark_iv']=df_1['IV']/100
    
    df_1.to_csv('Results/option_' + datetime.datetime.strftime(curr_day_starttime, '%Y%m%d') + '.csv')
    
    conf_fname = prepare_confidence_band_data(df_1)
    
    rdate = base.as_Date(curr_day.strftime('%Y-%m-%d'))
    rdate_f = base.format(rdate, '%Y-%m-%d')
    
    tau = df['tau'].unique()[0]
    
    spd_btc = bootstrap(conf_fname, 'Data/BTC_USD_Quandl_2015_2022.csv', rdate_f, tau, simmethod, r, 
                                     'Results/Rookley_Q_CB_1722_1_2_5_1_ttm39', r_bandwidth, df_1['moneyness'].min(), df_1['moneyness'].max())
    
    try :
        
        spd_btc.to_csv('Results/Rookley_Q_CB_1722_1_2_5_1_ttm39/btc_pk_'  + str(curr_day_starttime.__format__('%Y-%m-%d')) +'_bw_' + str(r_bandwidth) +  '.csv')
        
        print(spd_btc)
        
    except Exception as e:
        error_i.append(i0)

Using Macos Locale


/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
R[write to console]: 
Attaching package: ‘matlab’


R[write to console]: The following object is masked from ‘package:stats’:

    reshape


R[write to console]: The following objects are masked from ‘package:utils’:

    find, fix


R[write to console]: The following object is masked from ‘package:base’:

    sum


R[write to console]: KernSmooth 2.23 loaded
Copyright M. P. Wand 1997-2009

2017-08-21 00:00:00


/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_27951/3932245426.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_27951/3932245426.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_27951/3932245426.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] "bandwidth: 0.3"


...

[1] 44.694713  4.722982 66.258718
                [,1]      [,2]         [,3]        [,4]
sigmas -1.2831283850 1.5345826 -2.409455026  4.06876413
       -1.2494109453 1.5112903 -2.367986033  4.03045995
       -1.2167933870 1.4884171 -2.326968704  3.99242902
       -1.1852062135 1.4659580 -2.286400193  3.95467392
       -1.1545863141 1.4439082 -2.246277534  3.91719701
       -1.1248762043 1.4222628 -2.206597654  3.88000046
       -1.0960233763 1.4010170 -2.167357367  3.84308620
       -1.0679797411 1.3801660 -2.128553378  3.80645593
       -1.0407011457 1.3597051 -2.090182289  3.77011112
       -1.0141469550 1.3396295 -2.052240600  3.73405299
       -0.9882796880 1.3199343 -2.014724710  3.69828251
       -0.9630646992 1.3006150 -1.977630927  3.66280038
       -0.9384699001 1.2816668 -1.940955467  3.62760705
       -0.9144655132 1.2630850 -1.904694461  3.59270271
       -0.8910238552 1.2448651 -1.868843960  3.55808727
       -0.8681191456 1.2270025 -1.833399939  3.52376036
       -0.8457

        0.4982597260 0.7478336  0.363348079  0.14299196
        0.5040180855 0.7513164  0.364541405  0.11700555
        0.5097434760 0.7548095  0.365482751  0.09097920
        0.5154362729 0.7583105  0.366171588  0.06491282
        0.5210968453 0.7618170  0.366607386  0.03880631
        0.5267255559 0.7653265  0.366789611  0.01265956
        0.5323227613 0.7688366  0.366717728 -0.01352749
        0.5378888123 0.7723449  0.366391202 -0.03975495
        0.5434240538 0.7758488  0.365809494 -0.06602288
        0.5489288250 0.7793460  0.364972065 -0.09233135
        0.5544034595 0.7828338  0.363878373 -0.11868042
        0.5598482855 0.7863100  0.362527878 -0.14507016
        0.5652636258 0.7897719  0.360920037 -0.17150061
        0.5706497981 0.7932170  0.359054304 -0.19797181
        0.5760071149 0.7966429  0.356930137 -0.22448379
        0.5813358838 0.8000471  0.354546989 -0.25103660
        0.5866364073 0.8034270  0.351904317 -0.27763024
        0.5919089833 0.8067802  0.349001574 -0.3

/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_27951/3932245426.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_27951/3932245426.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_27951/3932245426.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]         [,3]        [,4]
sigmas -0.643732591 0.9928205 -1.573325756  2.76091253
       -0.634106588 0.9850012 -1.560848533  2.76713649
       -0.624572362 0.9772407 -1.548317611  2.77322764
       -0.615128180 0.9695392 -1.535733655  2.77918481
       -0.605772356 0.9618971 -1.523097332  2.78500676
       -0.596503254 0.9543147 -1.510409308  2.79069221
       -0.587319279 0.9467924 -1.497670248  2.79623982
       -0.578218882 0.9393304 -1.484880821  2.80164821
       -0.569200557 0.9319290 -1.472041694  2.80691591
       -0.560262835 0.9245886 -1.459153535  2.81204144
       -0.551404289 0.9173095 -1.446217017  2.81702322
       -0.542623528 0.9100919 -1.433232811  2.82185964
       -0.533919199 0.9029363 -1.420201593  2.82654900
       -0.525289981 0.8958427 -1.407124044  2.83108958
       -0.516734590 0.8888116 -1.394000845  2.83547955
       -0.508251774 0.8818432 -1.380832684  2.83971707
       -0.499840311 0.8749378 -

        0.240801122 0.5495572  0.229923226  1.10045339
        0.244787000 0.5507887  0.235449905  1.06749485
        0.248757054 0.5520474  0.240790435  1.03430950
        0.252711408 0.5533325  0.245943480  1.00089974
        0.256650188 0.5546427  0.250907723  0.96726792
        0.260573514 0.5559772  0.255681864  0.93341637
        0.264481508 0.5573348  0.260264617  0.89934732
        0.268374289 0.5587146  0.264654714  0.86506300
        0.272251975 0.5601154  0.268850900  0.83056556
        0.276114683 0.5615362  0.272851935  0.79585710
        0.279962528 0.5629760  0.276656592  0.76093966
        0.283795623 0.5644336  0.280263657  0.72581526
        0.287614082 0.5659080  0.283671929  0.69048582
        0.291418015 0.5673982  0.286880218  0.65495323
        0.295207534 0.5689029  0.289887344  0.61921934
        0.298982746 0.5704212  0.292692138  0.58328591
        0.302743760 0.5719519  0.295293440  0.54715467
        0.306490681 0.5734940  0.297690099  0.51082728
        0.

In [2]:
print(error_i)

[0, 4, 8, 13, 16, 17]
